<a href="https://colab.research.google.com/github/jianfeng-xiyue/fengtree/blob/master/deepseek-train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%capture
# 这是一个 Jupyter Notebook 的魔法命令，用于隐藏命令的输出，让笔记本界面更整洁。

# 安装 unsloth 包。unsloth 是一个用于微调大型语言模型（LLM）的工具，可以让模型运行更快、占用更少内存。
!pip install unsloth

# 卸载当前已安装的 unsloth 包（如果已安装），然后从 GitHub 的源代码安装最新版本。
# 这样可以确保我们使用的是最新功能和修复。
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# 安装 bitsandbytes 和 unsloth_zoo 包。
# bitsandbytes 是一个用于量化和优化模型的库，可以帮助减少模型占用的内存。
# unsloth_zoo 可能包含了一些预训练模型或其他工具，方便我们使用。
!pip install bitsandbytes unsloth_zoo

In [9]:
from unsloth import FastLanguageModel  # 导入FastLanguageModel类，用来加载和使用模型
import torch  # 导入torch工具，用于处理模型的数学运算

max_seq_length = 2048  # 设置模型处理文本的最大长度，相当于给模型设置一个“最大容量”
dtype = None  # 设置数据类型，让模型自动选择最适合的精度
load_in_4bit = True  # 使用4位量化来节省内存，就像把大箱子压缩成小箱子

# 加载预训练模型，并获取tokenizer工具
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Qwen-1.5B",  # 指定要加载的模型名称
    max_seq_length=max_seq_length,  # 使用前面设置的最大长度
    dtype=dtype,  # 使用前面设置的数据类型
    load_in_4bit=load_in_4bit,  # 使用4位量化
    # token="hf_...",  # 如果需要访问授权模型，可以在这里填入密钥
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.1: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.81G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

In [10]:
prompt_style = """以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是电气工程师，精通各类电器件诊断。
请回答以下算命问题。

### 问题：
{}

### 回答：
<think>{}"""
# 定义提示风格的字符串模板，用于格式化问题

question = "CR21的器件判断参数是什么？"
# 定义具体的算命问题

In [11]:
FastLanguageModel.for_inference(model)
# 准备模型以进行推理

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")
# 使用 tokenizer 对格式化后的问题进行编码，并移动到 GPU

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
# 使用模型生成回答

response = tokenizer.batch_decode(outputs)
# 解码模型生成的输出为可读文本

print(response[0])
# 打印生成的回答部分

<｜begin▁of▁sentence｜>以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是电气工程师，精通各类电器件诊断。
请回答以下算命问题。

### 问题：
CR21的器件判断参数是什么？

### 回答：
<think>
好，我现在要回答关于CR21器件判断参数的问题。首先，CR21是一种常见的二极管，常用于指示电路中。它在工作时会有不同的状态，比如工作状态和断开状态，分别对应不同的指示。

在工作状态时，CR21会发出“NO”信号，表示有电流通过它。而当断开状态时，信号会变成“NO”的否定，也就是“YES”，表示没有电流，可能在电路中被用作断路器或指示灯。

另外，CR21的电流输出特性也是重要的。在工作状态下，电流会增加，而在断开状态下，电流会减少甚至为零。这一点在电路中可以帮助判断器件的状态，尤其是在动态电路中，可能需要及时检测电流变化以确保器件的正常工作或断开。

还有，CR21的电阻特性也很重要。在工作状态下，电阻值较高，而在断开状态下，电阻值较低。这可能与电路中的负载和限流电路有关，特别是在需要动态控制的情况下，CR21可以作为限流器的一部分。

总结一下，CR21的判断参数主要涉及其工作状态、断开状态的指示，以及电流输出和电阻特性的变化，这些都是在电气工程师进行电路诊断时需要考虑的关键因素。
</think>

CR21是一种常见的二极管，常用于指示电路中。它在工作时会有不同的状态，分别对应不同的指示。

在工作状态时，CR21会发出“NO”信号，表示有电流通过它。而当断开状态时，信号会变成“YES”，表示没有电流，可能在电路中被用作断路器或指示灯。

另外，CR21的电流输出特性也是重要的。在工作状态下，电流会增加，而在断开状态下，电流会减少甚至为零。这一点在电路中可以帮助判断器件的状态，尤其是在动态电路中，可能需要及时检测电流变化以确保器件的正常工作或断开。

还有，CR21的电阻特性也很重要。在工作状态下，电阻值较高，而在断开状态下，电阻值较低。这可能与电路中的负载和限流电路有关，尤其是在需要动态控制的情况下，CR21可以作为限流器的一部分。

总结一下，CR21的判断参数主要涉及其工作状态、断开状态的

加载数据集

In [12]:
# 定义一个用于格式化提示的多行字符串模板
train_prompt_style = """以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是电气工程师，精通各类电器件诊断。
请回答以下算命问题。

### 问题：
{}

### 回答：
<思考>
{}
</思考>
{}"""

In [13]:
# 定义结束标记（EOS_TOKEN），用于指示文本的结束
EOS_TOKEN = tokenizer.eos_token  # 必须添加结束标记

# 导入数据集加载函数
from datasets import load_dataset
# 加载指定的数据集，选择中文语言和训练集的前500条记录
dataset = load_dataset("jian2008/test", 'default', split = "train[0:16]", trust_remote_code=True)
# 打印数据集的列名，查看数据集中有哪些字段
print(dataset.column_names)

README.md:   0%|          | 0.00/89.0 [00:00<?, ?B/s]

R1.json:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20 [00:00<?, ? examples/s]

['Question', 'Response', 'Complex_CoT']


In [14]:
# 定义一个函数，用于格式化数据集中的每条记录
def formatting_prompts_func(examples):
    # 从数据集中提取问题、复杂思考过程和回答
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []  # 用于存储格式化后的文本
    # 遍历每个问题、思考过程和回答，进行格式化
    for input, cot, output in zip(inputs, cots, outputs):
        # 使用字符串模板插入数据，并加上结束标记
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)  # 将格式化后的文本添加到列表中
    return {
        "text": texts,  # 返回包含所有格式化文本的字典
    }

dataset = dataset.map(formatting_prompts_func, batched = True)
dataset["text"][0]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

'以下是描述任务的指令，以及提供进一步上下文的输入。\n请写出一个适当完成请求的回答。\n在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。\n\n### 指令：\n你是电气工程师，精通各类电器件诊断。\n请回答以下算命问题。\n\n### 问题：\n你是谁\n\n### 回答：\n<思考>\n 我是中国中车研发的屏柜诊断助手 \n\n</思考>\n我是中国中车研发的屏柜诊断助手<｜end▁of▁sentence｜>'

第五步 执行微调

In [15]:
FastLanguageModel.for_training(model)

model = FastLanguageModel.get_peft_model(
    model,  # 传入已经加载好的预训练模型
    r = 16,  # 设置 LoRA 的秩，决定添加的可训练参数数量
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",  # 指定模型中需要微调的关键模块
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,  # 设置 LoRA 的超参数，影响可训练参数的训练方式
    lora_dropout = 0,  # 设置防止过拟合的参数，这里设置为 0 表示不丢弃任何参数
    bias = "none",    # 设置是否添加偏置项，这里设置为 "none" 表示不添加
    use_gradient_checkpointing = "unsloth",  # 使用优化技术节省显存并支持更大的批量大小
    random_state = 3407,  # 设置随机种子，确保每次运行代码时模型的初始化方式相同
    use_rslora = False,  # 设置是否使用 Rank Stabilized LoRA 技术，这里设置为 False 表示不使用
    loftq_config = None,  # 设置是否使用 LoftQ 技术，这里设置为 None 表示不使用
)

Unsloth 2025.3.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [16]:
from trl import SFTTrainer  # 导入 SFTTrainer，用于监督式微调
from transformers import TrainingArguments  # 导入 TrainingArguments，用于设置训练参数
from unsloth import is_bfloat16_supported  # 导入函数，检查是否支持 bfloat16 数据格式

trainer = SFTTrainer(  # 创建一个 SFTTrainer 实例
    model=model,  # 传入要微调的模型
    tokenizer=tokenizer,  # 传入 tokenizer，用于处理文本数据
    train_dataset=dataset,  # 传入训练数据集
    dataset_text_field="text",  # 指定数据集中文本字段的名称
    max_seq_length=max_seq_length,  # 设置最大序列长度
    dataset_num_proc=2,  # 设置数据处理的并行进程数
    packing=False,  # 是否启用打包功能（这里设置为 False，打包可以让训练更快，但可能影响效果）
    args=TrainingArguments(  # 定义训练参数
        per_device_train_batch_size=2,  # 每个设备（如 GPU）上的批量大小
        gradient_accumulation_steps=4,  # 梯度累积步数，用于模拟大批次训练
        warmup_steps=5,  # 预热步数，训练开始时学习率逐渐增加的步数
        max_steps=75,  # 最大训练步数
        learning_rate=2e-4,  # 学习率，模型学习新知识的速度
        fp16=not is_bfloat16_supported(),  # 是否使用 fp16 格式加速训练（如果环境不支持 bfloat16）
        bf16=is_bfloat16_supported(),  # 是否使用 bfloat16 格式加速训练（如果环境支持）
        logging_steps=1,  # 每隔多少步记录一次训练日志
        optim="adamw_8bit",  # 使用的优化器，用于调整模型参数
        weight_decay=0.01,  # 权重衰减，防止模型过拟合
        lr_scheduler_type="linear",  # 学习率调度器类型，控制学习率的变化方式
        seed=3407,  # 随机种子，确保训练结果可复现
        output_dir="outputs",  # 训练结果保存的目录
        report_to="none",  # 是否将训练结果报告到外部工具（如 WandB），这里设置为不报告
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/16 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/16 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/16 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/16 [00:00<?, ? examples/s]

In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 16 | Num Epochs = 38
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 75
 "-____-"     Number of trainable parameters = 18,464,768


Step,Training Loss
1,3.130500
2,2.856400
3,2.871500
4,3.044400
5,2.861100
6,2.820600
7,2.675100
8,2.629600
9,2.568400
10,2.363100


Step,Training Loss
1,3.130500
2,2.856400
3,2.871500
4,3.044400
5,2.861100
6,2.820600
7,2.675100
8,2.629600
9,2.568400
10,2.363100


In [18]:
print(question) # 打印前面的问题

CR21的器件判断参数是什么？


In [23]:
question = "CK21线圈电压72V，触点电流为130mA，判断CK21的状态"
# 将模型切换到推理模式，准备回答问题
FastLanguageModel.for_inference(model)

# 将问题转换成模型能理解的格式，并发送到 GPU 上
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# 让模型根据问题生成回答，最多生成 4000 个新词
outputs = model.generate(
    input_ids=inputs.input_ids,  # 输入的数字序列
    attention_mask=inputs.attention_mask,  # 注意力遮罩，帮助模型理解哪些部分重要
    max_new_tokens=4000,  # 最多生成 4000 个新词
    use_cache=True,  # 使用缓存加速生成
)

# 将生成的回答从数字转换回文字
response = tokenizer.batch_decode(outputs)

# 打印回答
print(response[0])

<｜begin▁of▁sentence｜>以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是电气工程师，精通各类电器件诊断。
请回答以下算命问题。

### 问题：
CK21线圈电压72V，触点电流为130mA，判断CK21的状态

### 回答：
<think>
好的，我现在需要分析用户的问题，并提供一个准确的回答。用户提供的指令是作为电气工程师，精通各类电器件诊断，来回答算命问题。问题是中国电表CK21的线圈电压72V和触点电流130mA，判断其状态。

首先，我应该明确CK21这个器件的基本参数。根据文档2，CK21是一种RZDR二极圈，属于EMOD系列继电器。根据文档1，EMOD系列的OprtVol和OprtCurrt参数如下：

- CK11: OprtVol=63V, OprtCurrt=130mA
- CK22: OprtVol=66V, OprtCurrt=130mA
- CK21: OprtVol=72V, OprtCurrt=130mA
- CK33: OprtVol=75V, OprtCurrt=130mA

从参数可以看出，CK21的OprtVol是72V，OprtCurrt是130mA。

根据文档1的规则2.2，RZDR二极圈的OprtVol决定了继电器的吸合电压，OprtCurrt是触点电流，用于诊断过流或过压情况。根据文档1的规则2.1，OprtVol=72V对应CK11的75V，OprtCurrt=130mA对应CK11的130mA。因此，CK21应该处于吸合状态（OprtVol=72V）。

综上所述，CK21的状态应该是OprtVol=72V，触点电流=130mA，处于吸合状态。
</think>

CK21的状态是OprtVol=72V，触点电流=130mA，处于吸合状态。<｜end▁of▁sentence｜>
